## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [125]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [126]:
import yaml

In [127]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [128]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [129]:
saut="\n"

### Préparation des matrices de traits

In [130]:
features.add_config('bdlexique.ini')

In [131]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [132]:
genDigraphe=False
genGraphe=False

In [133]:
sousDossier="2015-Data/"
rulesDossier=""
#rulesDossier="MGC-160104-01-N-VerbesActions/"
samplePrefix="MGC-160104-"
sampleName=sousDossier+samplePrefix+"derivation2-SILVER"+".csv"
rulesPrefix=samplePrefix
rulesName=sousDossier+rulesDossier+rulesPrefix+"derivation2-02-N-SILVER-Regles"+".pkl"
if debug:
    logfile_name=analysisPrefix+"-"+samplePrefix+".log"
    logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

### Préparation des cases du paradigme

In [134]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [135]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [136]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [137]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [138]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [139]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [140]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [141]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [142]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [143]:
paradigmes=pd.read_csv(sampleName,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [144]:
phonologicalMap="-N"
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [145]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [146]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [147]:
paradigmes.stack().value_counts(dropna=True).sum()

4027

In [148]:
with open(rulesName, 'rb') as input:
    resultatsLecture = pickle.load(input)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [149]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

Attention l'analyse ne comprend pas toutes les cases


In [150]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [151]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [152]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [153]:
def generateParadigms(generation1,genDigraphe=True):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

In [154]:
def generate(lexeme,genDigraphe=True):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [155]:
len(paradigmes.dropna(thresh=2)["lexeme"])

150

In [156]:
paradigmes.dropna(thresh=2).count().sum()-paradigmes.dropna(thresh=2)["lexeme"].count()

3877

#### Calculer le score de la clique

In [157]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [158]:
#%%time
debug=False
#listeTest=[u"manger",u"boire",u"dormir",u"aller",u"neiger"]
#listeTest=paradigmes.dropna(thresh=2)["lexeme"].values.tolist()
listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
#listeTest=[u"abasourdir",u"évacuer"]
nbVerbes=len(listeTest)
print (nbVerbes)
globDigraphe=nx.DiGraph()
globGraphe=nx.Graph()
cliques=[]
cliquesScores={}
cliquesListes={}
numClique=0
progressBar = FloatProgress(min=0, max=nbVerbes)
display(progressBar)
for i,element in enumerate(listeTest):
#    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
    progressBar.value=i
    result=generate(element,genDigraphe)
    (generation,lexDigraphe,lexGraphe,lexCliques)= result
#    print (generation,lexDigraphe,lexGraphe,lexCliques)
    if genDigraphe:
        globDigraphe=nx.union(globDigraphe,lexDigraphe)
    if genGraphe:
        globGraphe=nx.union(globGraphe,lexGraphe)
    cliques.extend(lexCliques)
    for clique in lexCliques:
        cliquesScores[numClique]=cliqueScore(clique,lexGraphe)
        cliquesListes[numClique]=clique
        numClique+=1

150


In [159]:
%%time
versionStamp=dateheure()
if genDigraphe: 
    nx.write_dot(globDigraphe,u"2015-Data/digraphe-%s.dot"%versionStamp)
if genGraphe:
    nx.write_dot(globGraphe,u"2015-Data/graphe-%s.dot"%versionStamp)

CPU times: user 38 µs, sys: 13 µs, total: 51 µs
Wall time: 57.9 µs


In [1]:
print (len(cliques))

NameError: name 'cliques' is not defined

In [161]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

{1: 1,
 2: 164,
 3: 436,
 4: 800,
 5: 743,
 6: 1208,
 7: 1494,
 8: 1770,
 9: 2064,
 10: 2108,
 11: 2056,
 12: 2062,
 13: 1889,
 14: 2215,
 15: 1734,
 16: 1522,
 17: 1232,
 18: 1090,
 19: 927,
 20: 982,
 21: 642,
 22: 666,
 23: 523,
 24: 524,
 25: 412,
 26: 387,
 27: 383,
 28: 328,
 29: 295,
 30: 255,
 31: 215,
 32: 254,
 33: 271,
 34: 209,
 35: 178,
 36: 157,
 37: 99,
 38: 95,
 39: 38,
 40: 37,
 41: 32,
 42: 18,
 43: 23,
 44: 3,
 45: 2,
 46: 2}

### Faire la liste des cases lexicalisées de l'échantillon

In [162]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])

## Comparer la sortie des cliques avec le paradigme de départ

In [163]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
progressBar = FloatProgress(min=0, max=len(cliques))
display(progressBar)
for n,clique in enumerate(cliques):
    progressBar.value=n
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
        point=clique[0].split("-")
        lPoint=len(point)
        if lPoint==3:
            lexeme=point[0]
        else:
            lexeme="-".join(point[0:len(point)-2])
        paradigmeClique["lexeme"]=lexeme
        casesLexeme=casesLexicales[lexeme]
        nbInitial=len(casesLexeme)
        if casesLexeme and len(casesLexeme)<=len(clique):
            for element in casesLexeme:
                champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
                if ","  in champForme:
                    formes=champForme.split(",")
                    nbInitial+=len(formes)-1
                    okFormes=False
                    for forme in formes:
                        pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                        if pointCase in clique:
                            okFormes=True
                            if debug: print ("point",pointCase)
#                            if debug: print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                    if okFormes:
                        sampleOK=True
                    else:
                        sampleOK=False
                        break
                else:
                    forme=champForme
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    if debug: print (pointCase, clique)
                    if not pointCase in clique:
                        sampleOK=False
                        break
        else:
            sampleOK=False
        if sampleOK:
            for element in clique:
                elements=element.split("-")
                forme=elements[-2]
                taminfo=elements[-1]
#                try:
#                 (lexeme,forme,taminfo)=element.split("-")
#                except ValueError:
#                    print (element)
                paradigmeClique[taminfo]=forme
#                if taminfo in paradigmes.columns:
#                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
#                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
#                            sampleOK=False
#                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
#            if n<100: print (clique==cliquesListes[n])
            paradigmeClique["score"]=cliquesScores[n]
            paradigmeClique["ajouts"]=len(clique)-nbInitial
#            print (cliqueScore(clique),clique)
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
#    else:
#        print ("break")
#        print (n,lexeme,clique)
#        break

#paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=[u"lexeme"]+sampleCases+[u"score",u"ajouts"])
#print (seuilClique)

CPU times: user 41.7 s, sys: 3.35 s, total: 45.1 s
Wall time: 48 s


In [164]:
paradigmesCLIQUES

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,FS,FP,score,ajouts
0,accroître,NaN,NaN,NaN,NaN,NaN,NaN,akrwatrô,akrwatrE,akrwatre,...,NaN,akrwat,NaN,akrwat,NaN,akrwat,NaN,NaN,178.560240,11
1,accroître,NaN,NaN,NaN,NaN,NaN,NaN,akrwatrô,akrwatrE,akrwatre,...,NaN,akrwat,NaN,akrwat,akrwat,akrwat,NaN,NaN,187.203177,11
2,agonir,NaN,NaN,NaN,NaN,agOnir,agOni,agOnirô,agOnirE,agOnire,...,NaN,NaN,NaN,NaN,NaN,NaN,agOni,agOni,216.316843,3
3,aller,alam,alE,NaN,NaN,alEr,ala,irô,irE,ire,...,NaN,aj,NaN,aj,aj,aj,ale,ale,917.026475,3
4,amener,NaN,am9nE,NaN,NaN,am9nEr,am9na,NaN,amEn9rE,amEn9re,...,NaN,NaN,NaN,amEn,NaN,amEn,NaN,NaN,391.409755,8
5,amener,NaN,am9nE,NaN,NaN,am9nEr,am9na,NaN,amEn9rE,NaN,...,NaN,NaN,NaN,NaN,NaN,amEn,NaN,NaN,318.826196,5
6,arriver,arivam,arivE,NaN,NaN,arivEr,ariva,ariv9rô,ariv9rE,ariv9re,...,arivjô,ariv,NaN,ariv,ariv,ariv,arive,arive,921.212568,6
7,asseoir,NaN,asi,NaN,asi,asir,asi,NaN,NaN,NaN,...,NaN,NaN,NaN,asEj,NaN,NaN,NaN,NaN,233.345136,5
8,atteindre,atEJim,atEJi,NaN,NaN,atEJir,atEJi,atêdrô,atêdrE,atêdre,...,NaN,atEJ,NaN,NaN,NaN,atEJ,atêt,atêt,431.971279,12
9,attendre,NaN,atâdi,NaN,NaN,atâdir,atâdi,NaN,atâdrE,atâdre,...,NaN,atâd,atâdje,atâd,atâd,atâd,atât,atât,523.729052,9


In [165]:
paradigmeMAX=paradigmesCLIQUES[paradigmesCLIQUES["score"]==paradigmesCLIQUES.groupby(["lexeme"])["score"].transform(max)].reset_index()
del paradigmeMAX["index"]
paradigmeMAX["score"]=paradigmeMAX["score"].apply(str)
paradigmeMAX["ajouts"]=paradigmeMAX["ajouts"].apply(str)
paradigmeSILVER=paradigmeMAX.groupby("lexeme").agg(lambda x: ",".join(list(set(x.dropna().values)))).reset_index()

In [166]:
paradigmeSILVER.to_csv(path_or_buf=sampleName.replace(".csv","-Silver.csv"),encoding="utf8",sep=";")
#GOLD.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Gold.csv',encoding="utf8",sep=";")

In [167]:
sampleName.replace(".csv","-Silver.csv")

'2015-Data/MGC-160104-derivation2-SILVER-Silver.csv'

In [168]:
#len(paradigmes.dropna(thresh=2)["lexeme"].values.tolist())

In [169]:
paradigmeSILVER

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,FS,FP,score,ajouts
0,accroître,,,,,,,akrwatrô,akrwatrE,akrwatre,...,,akrwat,,akrwat,akrwat,akrwat,,,187.20317736,11
1,agonir,,,,,agOnir,agOni,agOnirô,agOnirE,agOnire,...,,,,,,,agOni,agOni,216.316842964,3
2,aller,alam,alE,,,alEr,ala,irô,irE,ire,...,,aj,,aj,aj,aj,ale,ale,917.026475302,3
3,amener,,am9nE,,,am9nEr,am9na,,amEn9rE,amEn9re,...,,,,amEn,,amEn,,,391.409754838,8
4,arriver,arivam,arivE,,,arivEr,ariva,ariv9rô,ariv9rE,ariv9re,...,arivjô,ariv,,ariv,ariv,ariv,arive,arive,921.21256845,6
5,asseoir,,asi,,asi,asir,asi,,,,...,,,,asEj,,,,,233.345136486,5
6,atteindre,atEJim,atEJi,,,atEJir,atEJi,atêdrô,atêdrE,atêdre,...,,atEJ,,,,atEJ,atêt,atêt,431.971278638,12
7,attendre,,atâdi,,,atâdir,atâdi,,atâdrE,atâdre,...,,atâd,atâdje,atâd,atâd,atâd,atât,atât,523.729051934,9
8,battre,,bati,,,batir,bati,batrô,batrE,batre,...,,bat,,,,bat,baty,baty,352.272673038,12
9,buter,bytam,bytE,,,bytEr,byta,byt9rô,byt9rE,byt9re,...,bytjô,byt,,byt,byt,byt,,,894.084914963,7
